In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import json
import pandas as pd

In [14]:
lines = []
with open("annotated/annotated.json") as fin:
    for line in fin.readlines():
        lines.append(json.loads(line))

lines = pd.DataFrame(lines)

lines['sentence_id'] = lines.meta.apply(lambda x: x['sentenceId'])

lines = pd.DataFrame(lines[['text', 'annotations', 'sentence_id']])

label_lookup = {
    21: "simple mention",
    22: "mention with stereotype(s)",
    23: "mention with negative sentiment",
}

def parse_labels(label_list):
    parsed = []
    for label in label_list:
        parsed.append(
            label_lookup[label['label']]
        )
    return parsed

lines['annotations'] = lines.annotations.apply(parse_labels)
lines[lines.annotations.apply(len)>1]

,text,annotations,sentence_id
306,Aussi je brûle d'un vif désir de pénétrer en A...,"[mention with stereotype(s), mention with nega...",294
637,Le Charretier était muet - je vous rappelle qu...,"[simple mention, mention with stereotype(s)]",628
638,C'est Fritz qui devenait assommant; il tiraill...,"[mention with stereotype(s), mention with nega...",629
852,J'ai échappé à cette vermine de Fritz.,"[mention with stereotype(s), mention with nega...",845
965,Le sol est solide et Fritz est loin.,"[simple mention, mention with stereotype(s), m...",959
975,Les Boches ont cru à une attaque et ont répond...,"[simple mention, mention with negative sentiment]",969
1030,On rigole; on se dit que Fritz n'a plus un poi...,"[mention with stereotype(s), mention with nega...",1024
1254,Un violent ptt de 75 : c'est pour les cuistots...,"[mention with stereotype(s), mention with nega...",1248
1269,On riait en pensant à la débandade du pauvre F...,"[mention with stereotype(s), mention with nega...",1263
1434,Cologne nous parle tous les jours à deux heure...,"[simple mention, mention with stereotype(s)]",1428


In [15]:
from lxml import etree

parser=etree.XMLParser(remove_comments=True)
tree = etree.fromstring(open("outdir/sentencized_charles-destinataires.xml").read(), parser=parser)

In [33]:
def get_sentence_el(id_):
    result = tree.xpath(f"//tei:s[@id='{id_}']", namespaces={
        "tei":"http://www.tei-c.org/ns/1.0"
    })
    if len(result) == 1:
        return result[0]
    elif len(result) == 0:
        return None
        raise KeyError(f"id {id_} not found.")
    else:
        raise KeyError(f"multiple {id_} ids found.")

def get_type(el):
    type_ = None
    cel = el
    while type_ is None and cel.getparent() is not None:
        if 'type' in cel.attrib and cel.attrib['type'] == "journal":
            type_ = "Diary"
        elif 'type' in cel.attrib and cel.attrib['type'] == "note":
            type_ = "Note"
        elif 'type' in cel.attrib and cel.attrib['type'] == "lettre":
            if 'ana' in cel.attrib:
                type_ = cel.attrib['ana'].split(" ")[0].split("dest:")[1]
                if "T" in type_:
                    type_="Letter to Maurice Journé"
                else:
                    type_="Letter to Family"
            else:
                import pdb; pdb.set_trace()
        cel = cel.getparent()
    return type_

def get_chapter(el):
    chapter = None
    cel = el
    while chapter is None and cel.getparent() is not None:
        if 'type' in cel.attrib and cel.attrib['type'] == "chapter":
            chapter = cel.attrib['n']
        cel = cel.getparent()
    return chapter

In [17]:
#lines = pd.DataFrame(lines[lines.annotations.apply(len)>0])

lines['el'] = lines.sentence_id.apply(get_sentence_el)

lines = pd.DataFrame(lines[~lines['el'].isnull()])

In [34]:
lines['type'] = lines.el.apply(get_type)

lines['chapter'] = lines.el.apply(get_chapter)

In [36]:
lines = pd.DataFrame(lines[['text', 'annotations', 'sentence_id', 'type', 'chapter']])

lines.to_pickle("outdir/merged.pkl")

In [35]:
lines['type'].unique()

array(['Diary', 'Note', 'Letter to Family', None,
       'Letter to Maurice Journé'], dtype=object)

In [8]:
for (chapter, type_), subdf in lines.sort_values(["chapter", "type"]).groupby(["chapter", "type"]):
    print(chapter)
    print(type_)
    for _,row in subdf.iterrows():
        print(row.text)

1
note
Elle a été pour moi une surprise.
J'étais chez mon frère à Fresne St Mamès, dans la Haute-Saône, au moment de la mobilisation.
Quand les cloches, vers quatre heures de l'après-midi, ont sonné à toute volée, nous étions à la pêche.
Nous avons été atterrés, quoique mon frère fût indisponible et moi auxiliaire, invité à rester dans mes foyers en attendant des ordres ultérieurs.
Je répétais tout haut qu'après tout c'était la Revanche, la Revanche tant espérée, et qu'il n'y avait pas de quoi se désoler.
Notre sentiment et le sentiment des habitants du village, était un sentiment de grave inquiétude, toutefois sans fléchissement ni désarroi.
Je n'avais pas mon livret militaire.
Les gendarmes ne m'autorisaient pas à aller à Nancy,"qui était peut-être aux mains des Allemands".
Je m'en fus à Langres offrir mes services à un vieux commandant de recrutement qui me reçut très aimablement.
Il n'expliqua que la discipline militaire ne souffrait ni relâchement ni zèle, que chacun avait sa plac

16 - 10 - 16.-16h.- Cette nuit, bombardement d'avions.
Une nuit calme, remarquablement limpide.
Les aéros boches s'entendaient très distinctement, mais je n'ai jamais pu les apercevoir.
De 11h.à 3h.du matin, ils ont laché une cinquantaine de bombes, dont l'une est tombée à une cinquantaine de mètres de moi.
Au début, je veillais.
J'entendais à peu près les ronflements du moteur, à 7 ou 800m., si j'en juge par la netteté.
Puis les départs de deux batteries contre avions, les tac-tac des mitrailleuses, des coups de fusil isolés, et, je suppose, les éclatements des bombes.
Ceux-ci, assez lointains,n'étaient pas reconnaissables pour moi.
Je pensais qu'il y avait un seul avion allemand faisant sa ronde, cherchant son point et s'amusant à nous narguer.
Je me demandais aussi si ce n'était pas un avion français que j'entendais.
En tous cas j'ai pu remarquer avec quel à propos les mitrailleuses tiraient, quand l'avion ennemi cessait de se faire entendre.
Il était bien visible que tout ce monde-

Nous entendions les chocs assourdis des gros obus qui trouaient le sol au dessus de nos têtes.
L'un d'eux - un 210 - tomba sur bord d'un puits d'aération, tout contre le poste téléphonique, précipitant dans le creute plusieurs mètres cubes de terre, coupant toutes nos lignes.
Ce fut un beau gâchis.
C'était l'agitation d'un secteur après une grosse affaire.
Des opérations de détail, attaques et contre-attaques, se succédaient, insignifiantes.
L'une de ces contre-attaques boches avait été signalée par un déserteur.
A l'heure dite, les artilleurs français étaient à leurs piéces, et les hommes aux créneaux.
On laissa sortir les Boches, qui furent ensuite écrasés.
J.- Nos régiments, décimés, n'en pouvaient plus de fatigue.
Au lieu de nous relever, le commandement nous fit "glisser" vers la Gauche, entre Vailly et Condé, dans un secteur plus tranquille.
Je pars seul, à la suite d'un malentendu.
Je traverse Chavonne, rasé à ras de terre, Vailly abîmé, mais debout encore.
Je retrouve mes camar

La maison de mon étudiant, à Nomeny, n'existe plus.- D'ailleurs c'est très bizarre.
A Lunéville, le bureau d'enregistrement, abandonné, n'a été ni brûlé, ni pillé.
J'ai pris un bon du trésor de 500fr.
J'ai vu le collègue des Successions et l'inspecteur qui s'attend à partir.
Le contrôleur, VIDAL, est blessé légèrement au bras et dans le midi.
Le rédacteur,Bolleker, est sergent instructeur à Belfort.
L'inspecteur me demande l'adresse de COLAS pour tâcher de le rejoindre à Langres, s'il part.
Quelques machines, toutes les plus mauvaises, promènent des trains de voyageurs.
Je n'ai vu qu'un train de ravitaillement et un train vide.
Ça sent le danger.
Le canon vient de reprendre, toujours au loin.
Ça n'est pas gai.
Je ne peux pas trouver de charbon.
La ville est dans l'obscurité: on n'allume pas à cause des aviateurs allemands.
Tous les cafés sont fermés après sept heures.
Comme nouvelles, on est bien renseignés.
Je lis le Temps, qui est tout à fait bien,et, par ci par là, d'autres journaux

A Guerbigny, qui est tout contre, nous relevons la division coloniale.
Les sans-filistes sont gentiment installés dans une cave minuscule, à flanc de côteau.
Ils ont failli prendre le village d'assaut.
Ils y pénétraient bravement, par ordre,avec leur voiture, quand un agent de liaison s'est précipité pour les arrêter : ils avaient dépassé le bataillon d'attaque !
Nous montons notre antenne et prenons l'écoute permanente.
J'ai profité de ma veillée pour écrire.
J.- Pendant quelques jours, c'est la vie de secteur.
Pas un obus ne tombe sur le village.
Le soir, les échelons d'artillerie, de l'autre côté de la rivière, à quelques centaines de mètres, sont bombardés par avions.
J'assiste un soir à un réglage curieux.
Les avions boches, en l'air, demandent à coups de mitrailleuses deux obus de réglage à une batterie.
Les obus précisent l'emplacement aux gothas, qui lâchent alors leurs bombes.
Les Boches s'attaquent aussi a une batterie de 270 qui est derrière nous.
Un obus touche la voiture d

Il paraît que nous sommes dans une armée commandée par des imbéciles : ce serait grâce à eux que nous n'avons nos lettres que par hasard, et que nos marches ressemblent à celle d'un escargot ivre.
En tous cas, nous qui avons appartenu a une armée combattante, nous sommes un peu vexés de voir ces troupes de dixième ordre et ces généraux incapables s'offrir toutes les entrées triomphales.
La population est toujours parfaite avec nous de toute façon.
Partout nous avons des poêles et du bois.
24 - 11 - 18.- PM " Je vous écris de Hoerdt, environ dix kilomètres de Strasbourg.
Nous ne comptons pas y moisir, et demain nous remettons cela.
Rien de particulier à signaler.
Petite étape, dans le brouillard.
Pays rigoureusement plat : houblon, oeillette , asperges.
Plus de vignes.
La population me paraît moins emballée ici que dans les villages précédents.
On a décrété que le marc valait 0fr.70 : première sottise, dont les conséquences menacent de se développer.
Pétain est venu ici faire un petit t

Les Boches ont avancé contre nous en cet endroit.
A cent mètres, on voit une ligne de trous individuels qu'ils ont creusés, en profitant des arbres.
Ils y ont laissé des bandes, des chargeurs, des paquets de pansement.
Puis ils ont fait un bond de 25m. et l'on retrouve une nouvelle ligne de trous, avec les mêmes débris.
24 - 6 - 15.- T."... Hier après-midi, deux douzaines 105 percutants, un peu derrière nous.
Le soir, un petit choix composé d'un 105 percutant, d'une demi-douzaine de 105 fusants, de 2 ou 3 77 (autrement dits: obus de santé) fusants et d'un ou deux percutants.
Quelques échantillons de produits boches.
A part cela ce sont depuis Dimanche soir des bombardements effrayants.
D'abord nous autres.
Toutes les batteries crachent en pleine vitesse.
C'est un formidable roulement de tonnerre où l'on ne distingue plus ni départs, ni arrivées.
On entend juste d'affreux sifflements.
Ça peut durer dix minutes ou plusieurs heures.
Une fois les positions prises, les Boches commencent la 